# From policy functions to General Equilibrium

So far we solved for an individual's problem.
The main product of the solution is a (set of) policy functions.
However, we did not bring these policy functions to the general equilibrium of the economy.
This TA session is about filling this gap.
We use the paper of [Huggett (1993)](https://www.sciencedirect.com/science/article/pii/016518899390024M) as an example to do this.

This session is organized in two parts: first we see methodologically how to bring an individual's problem to General Equilibrium.
Second, we replicate Huggett (1993) and we summarize the methodology and the findings of the paper.

In [1]:
import numpy as np
from time import time
from scipy import optimize as opt
from matplotlib import pyplot as plt

## A simple exchange economy in equilibrium

Consider a riskless simple exchange economy where the representative household solves the following:

\begin{align*}
    \max_{c_t, a_{t+1}} &\; \sum_{t=0}^{\infty} \beta^t \dfrac{c_t^{1-\gamma}}{1-\gamma} \\
    \text{s.t.} &\; 
        \begin{cases}
            c_t + a_{t+1} \leq y_t + (1 + r_t) a_t \\
            a_{t+1} \geq - \mathcal{A} \\
            a_0 \text{ given},
        \end{cases}
\end{align*}

where $a_t$ is asset holdings that pay a net return $r_t$ and $\mathcal{A} > 0$ is the natural debt limit.
Note that here we allow for negative asset holdings, which we interpret as debt.


### Closed-form solution

In this economy, for given initial condition $a_0$ and endowment stream $\{ y_t \}_{t=0}^{\infty}$, we define the general equilibrium to be a sequence of allocations $\{ c_t^*, a_{t+1}^* \}_{t=0}^{\infty}$ and prices $\{ r_t^* \}_{t=0}^{\infty}$ such that there is no debt in the economy, that is $a_{t+1}^* = 0$ for all periods $t$.

The first-order conditions of the problem are

\begin{align*}
    \begin{cases}
        {\left( \dfrac{c_t}{c_{t+1}} \right)}^{-\gamma} = \beta (1 + r_{t+1}) \\
        c_t + a_{t+1} = y_t + (1 + r_t) a_t.
    \end{cases}
\end{align*}

In equilibrium, where $a_t = 0$, we have

\begin{align*}
    \begin{cases}
        r_{t+1}^* = \dfrac{1}{\beta} \cdot {\left( \dfrac{c_t^*}{c_{t+1}^*} \right)}^{-\gamma} - 1 \\
        c_t^* = y_t 
    \end{cases}
\end{align*}

This problem is not particularly interesting, economically speaking.
However, it is very simple to implement numerically.
The presence of a closed-form solution allows us to check the accuracy of the numerical outcome.


### Numerical solution

We take care of the numerical implementation by re-writing the problem in recursive form.

\begin{align*}
    V(a) = \max_{c(a), a'(a)} &\; \dfrac{c^{1-\gamma}}{1-\gamma} + \beta V(a') \\
    \text{s.t.} &\;
        \begin{cases}
            c + a' \leq y + (1 + r) a \\
            a' \geq - \mathcal{A}.
        \end{cases}
\end{align*}

We proceed by calibrating the model.
We set parameters $\beta$ and $\gamma$ to standard values in the literature.

In [2]:
beta = 0.95
gamma = 1.5

We express all the variables in the model relative to output by imposing $y_t = 1$ for all periods $t$.

In [3]:
y = 1.0

In principle, $\mathcal{A}$ should at least be equal to the natural debt limit (we're not interested in occasionally binding constraints here).
In practice, as we want the constraint not to be binding, we just set $\mathcal{A}$ to a huge number and we verify in the solution that the policy function $a'(a)$ does not exhibit constrained behavior on low values of $a$.
The way we code this limit is by creating a sufficiently broad grid of asset holdings.
The lowest value on the grid will be interpreted as the borrowing constraint.

In [4]:
n = 300 + 1
a = np.linspace(-5, 5, num=n)  # ensuring there's a value that is exactly zero, see later

Given the analytical solution of the model and the calibration, we know that the equilibrium interest rate is

In [5]:
rSol = 1 / beta - 1
print('r* = {:.5f}'.format(rSol))

r* = 0.05263


However, assume that for some reason we do not have the closed-form solution available.
This is the case with more complicated models (e.g., the baseline Neo-Keynesian model).
Therefore, we need to solve the problem numerically.

The way we solve the model is the following.
Observe that we can solve the individual's problem for a given value of the interest rate $r$.
This will return a policy function for asset holdings, which we denote here with $a'(a; r)$.
Let $z(r) \equiv a'(0; r)$ denote the excess asset demand function.
We pick $a'(0; r)$ because we defined our equilibrium as $a_t = 0$ for all $t$, essentially requiring a steady state where $a'(0; r) = 0$.
In equilibrium, we must have $z(r) = 0$.
For any guess of $r$, $z(r)$ will be generally different from zero.
What we do is to keep solving the problem with different guesses of $r$ until we find $z(r) = 0$.
At each step of this procedure, we improve the search of the right value of $r$ by using the excess asset holding demand.
If $z(r) > 0$, we have excess demand and we must lower $r$.
If $z(r) < 0$, we have insufficient demand and we must set a higher $r$.

Python's **classes** turn out to be useful in this context.
Let us set up an `Agent` that solves the problem.

In [6]:
class Agent:
    
    def __init__(self, beta, gamma, a, y):
        self.beta = beta
        self.gamma = gamma
        self.y = y
        self.a = a
        
    def solve(self, r, tol=1e-6):
        n = a.size
        v = np.zeros((n,1))
        v_new = np.zeros((n,1))
        dr = np.zeros((n,1), dtype=int)
        criterion = 1
        n_iter = 0
        t0 = time()
        while criterion > tol:
            n_iter += 1
            for i in range(n):
                c = self.y + self.a[i] * (1 + r) - self.a
                c[c<=0] = np.nan
                u = c ** (1 - self.gamma) / (1 - self.gamma)
                obj = u + self.beta * v[:, -1]
                v_new[i] = np.nanmax( obj )
                dr[i] = obj.tolist().index(v_new[i])
            v = np.block([v, v_new])
            criterion = np.max(np.abs(v[:, -1] - v[:, -2]))
        t1 = time()
        a_opt = self.a[dr]
        self.v = v
        print('VFI took {0:.3f} seconds, {1} iterations (r={2:.3f}%).'.format(t1-t0, n_iter, r*100))
        # c_opt = self.y + self.a * (1 + r) - a_opt
        return a_opt

There it is.
Everytime we call `Agent.solve(r)`, we solve the model with VFI for a given interest rate $r$.


### The point of projection methods

Now we pass to the heart of projection methods.
Remember that we defined the excess asset demand function as $z(r) \equiv a'(0; r)$.
Suppose that we have a guess for the equilibrium interest rate $r$ and denote it with $r^{(n)}$.

- If $z(r) > 0$, then $r^{(n)}$ is too high and $r^{(n+1)}$ needs to be lower
- If $z(r) < 0$, then $r^{(n)}$ is too low and $r^{(n+1)}$ needs to be higher
- If $z(r) = 0$, then $r^{(n)}$ is approximately equal to $r^*$ and we stop the search algorithm

In essence, we are finding the zero of the excess demand function $z(r)$, which we know is monotonic and increasing with $r$.
The module [`scipy.optimize`](https://docs.scipy.org/doc/scipy/reference/optimize.html) provides zero-finding routines.
Here we use the [`scipy.optimize.brentq`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.brentq.html#scipy.optimize.brentq) function to solve our problem.
This routine searches the zero of a continuous function $f(x)$ in a closed interval $[a, b]$, such that $f(a) \cdot f(b) < 0$.
We therefore need to provide bounds to the search algorithm.

In [7]:
rLo, rHi = np.array([0.75, 1.25]) * rSol

Now, we initialize the representative agent `ra` with the parameters of the model.

In [8]:
ra = Agent(beta, gamma, a, y)

In [9]:
where_a_is_zero = a.tolist().index(0)
z = lambda x: ra.solve(x)[where_a_is_zero]
rStar = opt.brentq(z, rLo, rHi)

VFI took 13.063 seconds, 286 iterations (r=3.947%).
VFI took 13.249 seconds, 284 iterations (r=6.579%).
VFI took 13.124 seconds, 286 iterations (r=5.526%).
VFI took 13.174 seconds, 287 iterations (r=5.226%).


We see that the zero-finding algorithm quickly found the solution, in the sense that it did not need to try many candidate values of `r` before finding the solution.
We can also verify here how close the numerical solution is to the closed-form one.

In [10]:
print('Closed-form solution: r = {:.10f}'.format(rSol))
print('  Numerical solution: r = {:.10f}'.format(rStar))

Closed-form solution: r = 0.0526315789
  Numerical solution: r = 0.0522556391


Quite close indeed.

This is a quick example just to showcase how projection methods work.
A remark here is that we could have made everything tremendously faster by using PFI.
With this example we already get a sense of how long projection methods might take.
We might need to solve the individual's problem many times before we can solve for the general equilibrium of a model.

## Huggett (1993)

    coming soon


### Summary and main results

    coming soon


### Replicating the paper

    coming soon